# Libraries

In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms
import torchvision.utils as vutils

import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image 
import imageio
from einops import rearrange 
from tqdm import tqdm 

import wandb

%matplotlib inline

In [2]:
print("CUDA available:", torch.cuda.is_available())
print("torch version:", torch.__version__)


CUDA available: False
torch version: 2.6.0+cpu


# Dataset

In [ ]:
from torchvision import CelebA

dataset = CelebA(root="data/", split="train", target_type="attr", download=True)